In [12]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [30]:
import pandas as pd
import numpy as np
import re

def clean_tweet(text):
    return re.sub("RT", "", text)

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv')
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv')
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv')
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv')
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, pp_cleaned_data, vox_cleaned_data])

display(data)

,tweet,label
0,"@JesusCintora: ""Sanitarios de la primaria de ...",__label__up
1,@DabitxoPiston: Donostia Scape Room,__label__up
2,@OskarMatute: Así se entiende mucho mejor el ...,__label__up
3,@IosuPosti: Hola! Mi nombre es Iosu Etayo Pal...,__label__up
4,"@OskarMatute: Pues, a riesgo de aburrir, lo r...",__label__up
...,...,...
4962,SE PODRA DECIR QUE EL ECHEMINGA ES UN DELINCUE...,__label__vox
4963,El colosal golpe de Herrera a Simancas por su ...,__label__vox
4964,"@MadamePerlan: @Macarena_Olona Hola Macarena,...",__label__vox
4965,@frandevox: Buenos días a todos desde la DlCT...,__label__vox


In [35]:
#Split data in training and test sets.
import csv
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.20, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training[training.columns.values[::-1]].to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

#Create test file.
test[test.columns.values[::-1]].to_csv('data.test', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

,tweet,label
4712,"@populares: 📺 @pablocasado_: ""Con la que está...",__label__pp
3369,"@Pacoboya: OPINIÓN | Un reto inaplazable, por...",__label__psoe
3100,"@SerafnMiguelGu1 @Apapell Dios, como estáis, ...",__label__psoe
1984,Siempre que sea una búsqueda y no una obsesión...,__label__up
4622,@vuthy1Iiving: Yo por ellas madre y ellas por...,__label__up
...,...,...
1314,@PPAlcobendas: Defendemos en #PlenoAlcobendas...,__label__pp
1994,@TvPlataforma: La nueva normalidad d ⁦@sanche...,__label__pp
411,@Correos: Este año celebramos el #12deOctubre...,__label__psoe
860,@PabloIglesias: Esta mañana ha llegado el pri...,__label__up


,tweet,label
3912,@MiaLorca: El gobierno tiene comprometido\n14...,__label__up
1510,Usar mascarilla es una medida importante para ...,__label__up
4709,"""El Pelos', Favorito a Liderar Podemos en Madr...",__label__vox
4159,@El_Dios_Odin1: Una fuente es la vendedora de...,__label__vox
3521,@miqueleandolini: @PhilAMellows Algo así.,__label__pp
...,...,...
2518,@mariaru25528317: @AngelG_A @DefensorPuebloE ...,__label__psoe
1333,@REDROJA4: Buenos días compañeros y compañera...,__label__up
4146,@mecompletocon: @EncuenPorLaRep Ahí estaremos...,__label__psoe
4801,Pedimos a @EU_Commission poder hacer aquí lo q...,__label__pp


In [80]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=25)

In [95]:
#To predict one instance.
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("Viva españa") )
print( model.predict("Pablo iglesias es el mejor") )
print( model.predict("Pablo iglesias es un mierdas") )
print( model.predict("La españa moderada") )

(('__label__vox',), array([1.0000093]))
(('__label__pp',), array([0.97923064]))
(('__label__vox',), array([0.77944636]))
(('__label__up',), array([0.96504098]))
(('__label__vox',), array([0.69973743]))
(('__label__up',), array([0.74366844]))
(('__label__psoe',), array([0.99863458]))


In [57]:
#To test the test set. (number of samples, precision, recall)
model.test("data.test")

precision = model.test("data.test")[1]
recall = model.test("data.test")[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Fscore:", F_score)

Fscore: 0.7094747423975873
